In [1]:
import ee
import os
ee.Initialize()

### Make a folder to hold fuel layers generated from your treatment scenario ensemble

In [2]:
scenarios_path = "projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios" # change this to given runs folder path
scenarios = ee.ImageCollection(scenarios_path)
#create a fuelscapes folder within the runs folder
fuels_folder=scenarios_path.replace('treatment_scenarios','fuelscapes_scenarios')
fuels_folder_list = os.popen(f"earthengine ls {os.path.dirname(scenarios_path)}").read().split('\n')[0:-1]
if not fuels_folder in fuels_folder_list:
    os.popen(f"earthengine create folder {fuels_folder}").read()
    print(f'Created Folder: {fuels_folder}')
else:
    print(f"{fuels_folder} already exists")

Created Folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios


### For each treated landscape img in your treatment scenarios imageCollection, create a new folder to hold the fuels layers

In [3]:
scenarios_img_paths = os.popen(f"earthengine ls {scenarios_path}").read().split('\n')[0:-1]
scn_sub_folders = [(fuels_folder+'/'+os.path.basename(path)) for path in scenarios_img_paths]

sub_folders_list = os.popen(f"earthengine ls {fuels_folder}").read().split('\n')[0:-1]
for sub_folder_name in scn_sub_folders:
    if not sub_folder_name in sub_folders_list:
        os.popen(f"earthengine create folder {sub_folder_name}").read()
        print(f'Created scenario fuelscapes sub-folder: {sub_folder_name}')
    else:
        print(f"{sub_folder_name} already exists")

Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario1
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario10
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario11
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario12
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario13
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario2
Created scenario fuelscapes sub-folder: projects/pyregence-ee/assets/aff

### Loop thru each fuel updater script the way we do for FireFactor: create_canopy_guide, calc_CC_CH, calc_CBD_CBH, calc_FM40 (don't need to run fm40 wui update)

In [4]:
# get config.yml file path, needed for every script
config_path = os.path.join(os.getcwd(),'config.yml')
print(config_path)

c:\aff-treatments\config.yml


### You can run Canopy Guide and FM40 updater scripts at same time since they do not depend on one another

In [5]:
# Canopy Guide
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cmd = f"python src/create_canopy_guide.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    #break

# FM40
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cmd = f"python src/calc_FM40.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    #break


python src/create_canopy_guide.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario1 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario1
python src/create_canopy_guide.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario10 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario10
python src/create_canopy_guide.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario11 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario11
python src/create_canopy_guide.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/a

### wait until all Canopy Guide exports have successfully completed before running the CC_CH script

In [6]:
# CC and CH
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cmd = f"python src/calc_CC_CH.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    print('\n')
    #break

python src/calc_CC_CH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario1 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario1


python src/calc_CC_CH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario10 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario10


python src/calc_CC_CH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario11 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario11


python src/calc_CC_CH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_1

### wait until all CC and CH exports have completed before running CBD_CBH 

In [7]:
# CBD and CBH
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    cmd = f"python src/calc_CBD_CBH.py -c {config_path} -d {scn_img_path} -o {scn_sub_folder}" # pass the config file path, the given scenarios DIST img path, and the given scenarios fuelscapes folder path
    print(cmd)
    os.popen(cmd).read()
    print('\n')
    #break

python src/calc_CBD_CBH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario1 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario1


python src/calc_CBD_CBH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario10 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario10


python src/calc_CBD_CBH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario11 -o projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario11


python src/calc_CBD_CBH.py -c c:\aff-treatments\config.yml -d projects/pyregence-ee/assets/aff-treatments/runs_sce

### After all fuel layers in each scenario's fuelscape has successfully exported, collate each 5-layer fuelscape into a multiband img for Google Drive Export

In [8]:
from src.utils.yml_params import get_export_params
AOI = ee.FeatureCollection("projects/pyregence-ee/assets/aff-treatments/PC442_2km_buffer_bounds").geometry()
for scn_img_path,scn_sub_folder in zip(scenarios_img_paths,scn_sub_folders):
    # print(scn_img_path)
    # print(scn_sub_folder)
    fm40 = ee.ImageCollection(scn_sub_folder+'/fm40_collection').select('new_fbfm40').mosaic()
    cc = ee.Image(scn_sub_folder+'/CC')
    ch = ee.Image(scn_sub_folder+'/CH')
    cbh = ee.Image(scn_sub_folder+'/CBH')
    cbd = ee.Image(scn_sub_folder+'/CBD')
    fuel_stack = fm40.addBands(cc).addBands(ch).addBands(cbh).addBands(cbd).rename('FM40','CC','CH', 'CBH','CBD').toInt16()
    # print(fuel_stack.bandNames().getInfo())

    # export
    runs_id = scn_sub_folder.split('/')[4]
    scn_id = scn_sub_folder.split('/')[-1]
    # print(runs_id)
    # print(scn_id)
    
    desc = f"export_{runs_id}_{scn_id}_fuelStack"
    crs,scale = get_export_params(config_path)
    # print(desc)
    # print(crs)
    # print(scale)
    folder=f'AFF_Fuelscapes'
    fileNamePrefix=f'{runs_id}_fuelStack_{scn_id}'
    
    task = ee.batch.Export.image.toDrive(image=fuel_stack,description=desc,folder=folder,fileNamePrefix=fileNamePrefix,region=AOI,scale=scale,crs=crs)
    task.start()
    print(f'Export started: {folder}/{fileNamePrefix}')
    #break

projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario1
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario1
['FM40', 'CC', 'CH', 'CBH', 'CBD']
runs_scenarios_13_distro_norm_20220919
scenario1
export_runs_scenarios_13_distro_norm_20220919_scenario1_fuelStack
EPSG:32611
30
Export started: AFF_Fuelscapes/runs_scenarios_13_distro_norm_20220919_fuelStack_scenario1
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/treatment_scenarios/scenario10
projects/pyregence-ee/assets/aff-treatments/runs_scenarios_13_distro_norm_20220919/fuelscapes_scenarios/scenario10
['FM40', 'CC', 'CH', 'CBH', 'CBD']
runs_scenarios_13_distro_norm_20220919
scenario10
export_runs_scenarios_13_distro_norm_20220919_scenario10_fuelStack
EPSG:32611
30
Export started: AFF_Fuelscapes/runs_scenarios_13_distro_norm_20220919_fuelStack_scenario10
projects/pyregence-ee/assets/aff-